In [ ]:
import sys
sys.path.append('/..')
from sociophysicsDataHandler import spdh
import numpy as np
import pandas as pd
# import requests
import pyarrow.parquet as pq
import glob

In [ ]:
data_handler = spdh.SociophysicsDataHandler()

In [ ]:
month = '03'
files = glob.glob(rf'C:\Users\caspouw\Downloads\EHV\04\01\EHV\Perron2.1\2*')
files[0]

In [ ]:
files = glob.glob(rf'C:\Users\caspouw\Downloads\EHV\04\01\EHV\Perron*\2*')
def convert_file_names(file):
#     print(f"--- Start file {file}")
    station = file.split('\\')[-3]
    file_name = file.split('\\')[-1]
    platform = file.split('\\')[-2]
#     platform_no = platform.split('.')[-1a
    date = file.split('\\')[-1].split('_')[0]
    if station.lower() == "ut":
        pf_indication = platform[-3]
        platform_name = 'Perron'
        platform_name2 = platform_name
    else:
        pf_indication = platform[-3:]
        platform_name = 'Platform'
        platform_name2 = platform_name.lower()
    return f"/ProRail_USE_LL_data/{station.lower()}/{platform_name2}{pf_indication}/{date[:8]}/{station}_{platform_name}{pf_indication}_{file_name}"

# for file in files:
#     print(convert_file_names(file))

In [ ]:
error_list = []
for station in ['EHV']:
# for station in ['UT']:
    for day in np.arange(26,31):
        print(f"Start day {day:02} station {station}")
#         files = glob.glob(rf'C:\Users\caspouw\Downloads\{month}\{day:02}\{station}\Perron*\*')
        files = glob.glob(rf'C:\Users\caspouw\Downloads\{station}\04\{day:02}\{station}\Perron*\*')
        for file in files:
            converted_file_name = convert_file_names(file)
            try:
#                 print('try')
                data_handler._SociophysicsDataHandler__oc_client.put_file(converted_file_name, 
                                                                          file)    
            except Exception as e:
#                 print('except')
#                 logging.error('Error at %s', 'division', exc_info=e)
#                 error_list.append(e)
                folder_name_parts = converted_file_name.split('/')
                if station == "ASDZ":
                    folder_name = f'/ProRail_USE_LL_data/{folder_name_parts[2]}/{folder_name_parts[3].lower()}/{folder_name_parts[4]}/'
                elif station == "UT":
                    folder_name = f'/ProRail_USE_LL_data/{folder_name_parts[2]}/{folder_name_parts[3]}/{folder_name_parts[4]}/'
                elif station == "EHV":
                    folder_name = f'/ProRail_USE_LL_data/{folder_name_parts[2]}/{folder_name_parts[3]}/{folder_name_parts[4]}/'
                data_handler._SociophysicsDataHandler__oc_client.mkdir(folder_name)
                print(f"Made folder {folder_name}")
                data_handler._SociophysicsDataHandler__oc_client.put_file(converted_file_name, file)    
        print(f"Finished day {day:02}")
        print("\n")

    complete_list = pd.DataFrame()
    folders = data_handler.list_files('/ProRail_USE_LL_data/').name.unique()
    stations = [s for s in folders if "_" not in s]
#     np.delete(stations, np.where(areas == 'eindhoven'))
    for station in stations:
        areas = data_handler.list_files(f'/ProRail_USE_LL_data/{station}/').name.unique()
        areas = [s for s in areas if ".pdf" not in s]
#         np.delete(areas, np.where(areas == 'eindhoven'))
        for area in areas:
            file_list = data_handler.list_files(f'/ProRail_USE_LL_data/{station}/{area}/')
            file_list['last_modified'] = file_list.attributes.apply(lambda x: list(x.values())[0])
            file_list = file_list[['name', 'last_modified']].copy()
            file_list.columns = ['date', 'last_modified']
            file_list['station'] = station
            file_list['area'] = area
            complete_list = complete_list.append(file_list)

    complete_list['last_modified'] = complete_list.last_modified.apply(lambda x: pd.to_datetime(x))
    complete_list = complete_list.sort_values('last_modified', ascending = False)
    complete_list[['station', 'area', 'date', 'last_modified']].to_csv('upload_log.csv')
    data_handler._SociophysicsDataHandler__oc_client.put_file(f"/ProRail_USE_LL_data/", "upload_log.csv")
    print('Log uploaded')

In [15]:
folder_name

'/ProRail_USE_LL_data/ehv/platform2.1/20210330/'

In [12]:
folder_name_parts

['',
 'ProRail_USE_LL_data',
 'ehv',
 'platform2.1',
 '20210330',
 'EHV_Platform2.1_2021033008_trajectorie.parquet']

In [9]:
converted_file_name

'/ProRail_USE_LL_data/ehv/platform2.1/20210330/EHV_Platform2.1_2021033008_trajectorie.parquet'

In [11]:
folder_name

NameError: name 'folder_name' is not defined